<a href="https://colab.research.google.com/github/vishal19975/vishal19975/blob/main/Next%20Character%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.utils.data_utils import get_file
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import SimpleRNN, TimeDistributed
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adam
import pickle as cPickle
import bcolz
import re
from numpy.random import random, permutation, randn, normal, uniform, choice

In [ ]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
len(text)


600893

In [ ]:
chars = sorted(list(set(text)))
len(chars)+1

85

In [ ]:
chars.insert(0, '\0')

In [ ]:
char_to_index = {v:i for i,v in enumerate(chars)}
index_to_char = {i:v for i,v in enumerate(chars)}

In [ ]:
total_index = [char_to_index[char] for char in text]

In [ ]:
total_index[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [ ]:
''.join(index_to_char[i] for i in total_index[:25])

'PREFACE\n\n\nSUPPOSING that '

In [ ]:
from past.builtins import xrange
pred_num = 7
xin = [[total_index[j+i] for j in xrange(0, len(total_index)-1-pred_num, pred_num)] for i in range(pred_num)]
y = [total_index[i+pred_num] for i in xrange(0, len(total_index)-1-pred_num, pred_num)]

In [ ]:
X = [np.stack(xin[i][:-2]) for i in range(pred_num)]
Y = np.stack(y[:-2])

In [ ]:
X

[array([40,  1, 39, ..., 67,  2,  2]),
 array([42,  1, 43, ...,  2, 54, 62]),
 array([29,  1, 33, ..., 76, 72, 67]),
 array([30, 43, 38, ..., 68,  2, 57]),
 array([25, 45, 31, ..., 71, 73, 62]),
 array([27, 40,  2, ..., 65, 61, 56]),
 array([29, 40, 73, ..., 57, 58, 54])]

In [ ]:
Y[:8]

array([ 1, 39, 61, 73,  2,  9, 61,  2])

In [ ]:
X[0].shape, Y.shape

((85839,), (85839,))

In [ ]:
hidden_layers = 256
vocab_size = 86
n_fac = 42

In [ ]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, activation='relu'),
        Dense(vocab_size, activation='softmax')
    ])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 7, 42)             3612      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 256)               76544     
                                                                 
 dense_1 (Dense)             (None, 86)                22102     
                                                                 
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
model.fit(np.stack(X, 1), Y, batch_size=64, epochs=5)

Epoch 1/5
1342/1342 [==============================] - 23s 17ms/step - loss: 2.5496
Epoch 2/5
1342/1342 [==============================] - 22s 17ms/step - loss: 2.0723
Epoch 3/5
1342/1342 [==============================] - 23s 17ms/step - loss: 1.8798
Epoch 4/5
1342/1342 [==============================] - 22s 16ms/step - loss: 1.7575
Epoch 5/5
1342/1342 [==============================] - 22s 16ms/step - loss: 1.6736


In [ ]:
model.save_weights('simpleRNN_3pred.h5')

In [ ]:
model.load_weights('simpleRNN_3pred.h5')

In [ ]:
model.save_weights('simpleRNN_7pred.h5')

In [ ]:
model.load_weights('simpleRNN_7pred.h5')

In [ ]:
def predict_next_char(inp):
    index = [char_to_index[i] for i in inp]
    arr = np.expand_dims(np.array(index), axis=0)
    prediction = model.predict(arr)
    return index_to_char[np.argmax(prediction)]

In [ ]:
predict_next_char('those w')

'h'

In [ ]:
ys = [[total_index[j+i] for j in xrange(1, len(total_index)-pred_num, pred_num)] for i in range(pred_num)]

In [ ]:
Y_return = [np.stack(ys[i][:-2]) for i in range(pred_num)]

In [ ]:
X

[array([40,  1, 39, ..., 67,  2,  2]),
 array([42,  1, 43, ...,  2, 54, 62]),
 array([29,  1, 33, ..., 76, 72, 67]),
 array([30, 43, 38, ..., 68,  2, 57]),
 array([25, 45, 31, ..., 71, 73, 62]),
 array([27, 40,  2, ..., 65, 61, 56]),
 array([29, 40, 73, ..., 57, 58, 54])]

In [ ]:
Y_return

[array([42,  1, 43, ...,  2, 54, 62]),
 array([29,  1, 33, ..., 76, 72, 67]),
 array([30, 43, 38, ..., 68,  2, 57]),
 array([25, 45, 31, ..., 71, 73, 62]),
 array([27, 40,  2, ..., 65, 61, 56]),
 array([29, 40, 73, ..., 57, 58, 54]),
 array([ 1, 39, 61, ...,  2,  2, 73])]

In [ ]:
vocab_size = 86
n_fac = 42
hidden_layers = 256

In [ ]:
return_model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, return_sequences=True, activation='relu'),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])

In [ ]:
return_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 7, 42)             3612      
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 7, 256)            76544     
                                                                 
 time_distributed (TimeDistr  (None, 7, 86)            22102     
 ibuted)                                                         
                                                                 
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [ ]:
return_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
X_model = np.stack(X, 1)
Y_model = np.expand_dims(np.stack(Y_return, 1), axis=-1)

In [ ]:
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

Epoch 1/5
1342/1342 [==============================] - 24s 17ms/step - loss: 2.3480
Epoch 2/5
1342/1342 [==============================] - 23s 17ms/step - loss: 1.9841
Epoch 3/5
1342/1342 [==============================] - 23s 17ms/step - loss: 1.8869
Epoch 4/5
1342/1342 [==============================] - 22s 17ms/step - loss: 1.8363
Epoch 5/5
1342/1342 [==============================] - 23s 17ms/step - loss: 1.8041


In [ ]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

Epoch 1/5
1342/1342 [==============================] - 23s 17ms/step - loss: 1.7472
Epoch 2/5
1342/1342 [==============================] - 23s 17ms/step - loss: 1.7353
Epoch 3/5
1342/1342 [==============================] - 23s 17ms/step - loss: 1.7300
Epoch 4/5
1342/1342 [==============================] - 23s 17ms/step - loss: 1.7258
Epoch 5/5
1342/1342 [==============================] - 22s 17ms/step - loss: 1.7222


In [ ]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

Epoch 1/5
1342/1342 [==============================] - 23s 17ms/step - loss: 1.7191
Epoch 2/5
1342/1342 [==============================] - 23s 17ms/step - loss: 1.7161
Epoch 3/5
1342/1342 [==============================] - 23s 17ms/step - loss: 1.7132
Epoch 4/5
1342/1342 [==============================] - 22s 17ms/step - loss: 1.7107
Epoch 5/5
1342/1342 [==============================] - 22s 16ms/step - loss: 1.7082


In [ ]:
return_model.save_weights('return_sequences_25.h5')

In [ ]:
def predict_every_char(inp):
    l = []
    p = 0
    while p<len(inp):
        pre_inp = inp[p:p+pred_num]
        if len(pre_inp) < pred_num:
            pre_inp = pre_inp + ' '*(pred_num - len(pre_inp))
            l.append(pre_inp)
        else:
            l.append(pre_inp) 
        p+=pred_num

#     index = [char_to_index[i] for i in inp]
#     arr = np.expand_dims(index, axis=0)
#     prediction = return_model.predict(arr)
#     return ''.join([index_to_char[np.argmax(i)] for i in prediction[0]])
    
    final = []
    for half in l:
        index = [char_to_index[i] for i in half]
        arr = np.expand_dims(index, axis=0)
        prediction = return_model.predict(arr)
        final.append(''.join([index_to_char[np.argmax(i)] for i in prediction[0]]))
    
    return ''.join(final)

In [ ]:
predict_every_char('and the boy left')

'nd the teu toa et    '

In [ ]:
predict_every_char('this is')

'hen is '